# Daily News

In [ ]:
# !pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 
# !pip install --upgrade google-auth-oauthlib
# !pip install openai

## Daily CNBC News

In [2]:
from YouTubeSummary import YouTubeSummary, CONFIG

# Use the default configuration
yt_summary = YouTubeSummary(CONFIG)
yt_summary.run()

2024-10-02 14:39:56,525 - INFO - Setting channel to CNBC_TV
2024-10-02 14:39:56,527 - INFO - Channel set to CNBC_TV (ID: UCrp_UI8XtuYfpiqluWLD7Lw)
2024-10-02 14:39:56,528 - INFO - Summaries will be saved in: /Users/sliu/src/mysrc/investing/razorback_investing/news/youtube_summary_data/CNBC_TV
2024-10-02 14:39:56,528 - INFO - Starting stage: _set_channel
2024-10-02 14:39:56,529 - INFO - Setting channel to CNBC_TV
2024-10-02 14:39:56,530 - INFO - Channel set to CNBC_TV (ID: UCrp_UI8XtuYfpiqluWLD7Lw)
2024-10-02 14:39:56,530 - INFO - Summaries will be saved in: /Users/sliu/src/mysrc/investing/razorback_investing/news/youtube_summary_data/CNBC_TV
2024-10-02 14:39:56,530 - INFO - Completed stage: _set_channel
2024-10-02 14:39:56,530 - INFO - Starting stage: _load_existing_data
2024-10-02 14:39:56,531 - INFO - No existing data found. Will start with an empty DataFrame.
2024-10-02 14:39:56,538 - INFO - Completed stage: _load_existing_data
2024-10-02 14:39:56,540 - INFO - Starting stage: _fetch

/Users/sliu/src/mysrc/investing/razorback_investing/news/fetch_videos.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, new_videos_df], ignore_index=True).sort_values(by=['Published At'], ascending=False)


An error occurred for video ID eFu-WlIXiUM: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=eFu-WlIXiUM! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
2024-10-02 14:40:31,880 - INFO - Saved videos with transcripts to /Users/sliu/src/mysrc/investing/razorback_investing/news/youtube_summary_data/CNBC_TV/youtube_summary_CNBC_TV_2024-10-02.csv
2024-10-02 14:40:31,881 - INFO - Completed stage: _add_transcripts
2024-10-02 14:40:31,883 - INFO - Starting stage: _generate_summaries
2024-10-02 14:40:31,886 - INFO - Generating summaries fo

## Youtube Channel Studies

In [ ]:
# Set custom channel and get all videos from that chanenl

# One-off study

In [ ]:
import fetch_videos
from YouTubeSummary import YouTubeSummary, CONFIG
import pandas as pd

# Specify the video IDs you want to study
video_ids = [
    "7GYLjlfbIIQ",
    # Add more video IDs as needed
    # "ANOTHER_VIDEO_ID",
    # "YET_ANOTHER_VIDEO_ID",
]

# Specify a single task to apply to all videos
task = """Provide a one-sentence summary of the video. Tell me the best products recomended by the videos"""

# Create a YouTubeSummary instance (for using its configuration)
yt_summary = YouTubeSummary(CONFIG)

# Function to process a single video with the task
def process_video(video_id, task):
    # Get transcript
    transcript = fetch_videos.get_transcript(video_id)
    
    # Apply the task
    result = fetch_videos.apply_task(transcript, yt_summary._client, task)
    
    return {
        "Video ID": video_id,
        "Transcript": transcript,
        "Task Result": result
    }

# Process each video
results = []
for video_id in video_ids:
    try:
        result = process_video(video_id, task)
        results.append(result)
        print(f"Processed video: {video_id}")
    except Exception as e:
        print(f"Error processing video {video_id}: {str(e)}")

# Create a DataFrame with the results
result_df = pd.DataFrame(results)

# Display results
for _, row in result_df.iterrows():
    print(f"Video ID: {row['Video ID']}")
    print(f"Transcript (first 500 characters): {row['Transcript'][:500]}...")
    print(f"Task Result: {row['Task Result']}")
    print("\n" + "="*50 + "\n")

# Optionally, save the results to a CSV file
# result_df.to_csv("one_off_study_results.csv", index=False)